### Links
- PyTorch MNIST example - https://github.com/pytorch/examples/blob/master/mnist/main.py
- pytorch cifar10 code - https://github.com/kuangliu/pytorch-cifar/blob/master/main.py
- LeNet for CIFAR-10 - https://github.com/kuangliu/pytorch-cifar/blob/master/models/lenet.py

### My notes
- My Google Doc - https://docs.google.com/document/d/1nPvFULraomLmBp4IuNVe51YXV1DQsSzgQWtuaZhbkjo/edit
- Trello - https://trello.com/b/ycCGjD3A/mobile-dl
- Using webcam - https://docs.google.com/document/d/17BVx5yioNaV11mip4U9gRAO7wLIJf-aOpIITJPjqeks/edit#
- Installing pytorch - https://docs.google.com/document/d/1NfLbuhoOxonzS9FEeooyqpB90-62Gat-uZ2DOTaWRdg/edit#


In [1]:
import argparse
import math
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
print(torch.__version__)
print(torch.__file__)

1.0.1.post2
/Users/william/.local/share/virtualenvs/william-oBc2a6gD/lib/python3.7/site-packages/torch/__init__.py


In [3]:
# Source: https://github.com/kuangliu/pytorch-cifar/blob/master/models/mobilenet.py

class Block(nn.Module):
    '''Depthwise conv + Pointwise conv'''
    def __init__(self, in_planes, out_planes, stride=1):
        super(Block, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, in_planes, kernel_size=3, stride=stride, padding=1, groups=in_planes, bias=False)
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv2 = nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
#         print(out.size())
        out = F.relu(self.bn2(self.conv2(out)))
#         print(out.size())
        return out


class MobileNet(nn.Module):
    # (128,2) means conv planes=128, conv stride=2, by default conv stride=1
    cfg = [(48,2), (64,2), (128,2), (256,2), 256]

    def __init__(self, num_classes=10):
        super(MobileNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layers = self._make_layers(in_planes=16)
        self.linear = nn.Linear(256, num_classes)

    def _make_layers(self, in_planes):
        layers = []
        for x in self.cfg:
            out_planes = x if isinstance(x, int) else x[0]
            stride = 1 if isinstance(x, int) else x[1]
            layers.append(Block(in_planes, out_planes, stride))
            in_planes = out_planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
#         print(out.size())
        out = self.layers(out)
#         print(out.size())
        out = F.avg_pool2d(out, 2)
#         print(out.size())
        out = out.view(out.size(0), -1)
#         print(out.size())
        out = self.linear(out)
#         print(out.size())
        out = F.log_softmax(out, dim=1)
        return out


In [4]:
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')


_StoreAction(option_strings=['--log-interval'], dest='log_interval', nargs=None, const=None, default=10, type=<class 'int'>, choices=None, help='how many batches to wait before logging training status', metavar='N')

In [ ]:
args = parser.parse_args(args=["--epochs=10"])
args

Namespace(batch_size=64, epochs=10, log_interval=10, lr=0.01, momentum=0.5, no_cuda=False, seed=1, test_batch_size=1000)

In [ ]:
use_cuda = not args.no_cuda and torch.cuda.is_available()
use_cuda

False

In [ ]:
torch.manual_seed(args.seed)

In [ ]:
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cpu')

In [ ]:
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
kwargs

{}

In [ ]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
#                        transforms.Resize((32, 32)),
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
#                        transforms.Resize((32, 32)),
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)


In [ ]:
model = MobileNet().to(device)
model

MobileNet(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): Block(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn2): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Block(
      (conv1): Conv2d(48, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=48, bias=False)
      (bn1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): B

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0.5
    nesterov: False
    weight_decay: 0
)

In [ ]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)  # to do: needed?
        optimizer.zero_grad()  # what does this do? zero-out any previous gradient info?
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            digits = int(math.log10(len(train_loader.dataset))) + 1
            print('Train Epoch: {} [{:05d}/{:05d} ({:02.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
            

In [ ]:
for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    test(args, model, device, test_loader)
    
    # save the model!
    # source for timestamp formatting: https://stackoverflow.com/q/10607688/781938
    timestamp = time.strftime("%Y%m%d-%H%M%S")
    # source for f-string: https://stackoverflow.com/a/42562236/781938
    torch.save(
        model.state_dict(),
        f'{timestamp}-mnist_mobilenet-{epoch:05d}.pytorchmodel')


Train Epoch: 1 [00000/60000 (00%)]	Loss: 2.314294
Train Epoch: 1 [00640/60000 (01%)]	Loss: 2.274302
Train Epoch: 1 [01280/60000 (02%)]	Loss: 2.236889
Train Epoch: 1 [01920/60000 (03%)]	Loss: 2.232794
Train Epoch: 1 [02560/60000 (04%)]	Loss: 2.171389
Train Epoch: 1 [03200/60000 (05%)]	Loss: 2.124286
Train Epoch: 1 [03840/60000 (06%)]	Loss: 2.053159
Train Epoch: 1 [04480/60000 (07%)]	Loss: 2.001033
Train Epoch: 1 [05120/60000 (09%)]	Loss: 2.012931
Train Epoch: 1 [05760/60000 (10%)]	Loss: 1.849557
Train Epoch: 1 [06400/60000 (11%)]	Loss: 1.801027
Train Epoch: 1 [07040/60000 (12%)]	Loss: 1.719778
Train Epoch: 1 [07680/60000 (13%)]	Loss: 1.623765
Train Epoch: 1 [08320/60000 (14%)]	Loss: 1.443497
Train Epoch: 1 [08960/60000 (15%)]	Loss: 1.523487
Train Epoch: 1 [09600/60000 (16%)]	Loss: 1.334521
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.290151
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.332378
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.308232
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.103068


Train Epoch: 2 [44160/60000 (74%)]	Loss: 0.093334
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.257949
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.037161
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.109451
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.060454
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.155622
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.120936
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.277414
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.053618
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.137496
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.194476
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.108682
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.073031
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.142928
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.105955
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.055439
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.277066
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.060142
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.078638
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.070262


Train Epoch: 4 [27520/60000 (46%)]	Loss: 0.023064
Train Epoch: 4 [28160/60000 (47%)]	Loss: 0.051470
Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.131223
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.080059
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.026607
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.010747
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.030527
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.062769
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.167022
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.057801
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.143760
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.122510
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.012872
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.088669
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.103156
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.059201
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.028176
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.077059
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.077546
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.011139


Train Epoch: 6 [10880/60000 (18%)]	Loss: 0.034987
Train Epoch: 6 [11520/60000 (19%)]	Loss: 0.026034
Train Epoch: 6 [12160/60000 (20%)]	Loss: 0.110125
Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.077922
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.096804
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.109125
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.056780
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.010773
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.033729
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.015654
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.010992
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.006759
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.022024
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.004606
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.028384
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.104811
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.005158
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.038401
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.018929
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.126295


Train Epoch: 7 [55040/60000 (92%)]	Loss: 0.010766
Train Epoch: 7 [55680/60000 (93%)]	Loss: 0.023478
Train Epoch: 7 [56320/60000 (94%)]	Loss: 0.011341
Train Epoch: 7 [56960/60000 (95%)]	Loss: 0.029457
Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.014379
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.014355
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.009341
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.066236

Test set: Average loss: 0.0442, Accuracy: 9852/10000 (99%)

Train Epoch: 8 [00000/60000 (00%)]	Loss: 0.010824
Train Epoch: 8 [00640/60000 (01%)]	Loss: 0.069814
Train Epoch: 8 [01280/60000 (02%)]	Loss: 0.007231
Train Epoch: 8 [01920/60000 (03%)]	Loss: 0.063903
Train Epoch: 8 [02560/60000 (04%)]	Loss: 0.039122
Train Epoch: 8 [03200/60000 (05%)]	Loss: 0.021219
Train Epoch: 8 [03840/60000 (06%)]	Loss: 0.008561
Train Epoch: 8 [04480/60000 (07%)]	Loss: 0.086360
Train Epoch: 8 [05120/60000 (09%)]	Loss: 0.072274
Train Epoch: 8 [05760/60000 (10%)]	Loss: 0.044488
Train Epoch: 8 [06400/60000 (11%)]	Loss

Train Epoch: 9 [38400/60000 (64%)]	Loss: 0.069140
Train Epoch: 9 [39040/60000 (65%)]	Loss: 0.071090
Train Epoch: 9 [39680/60000 (66%)]	Loss: 0.013832
Train Epoch: 9 [40320/60000 (67%)]	Loss: 0.056686
Train Epoch: 9 [40960/60000 (68%)]	Loss: 0.046584
Train Epoch: 9 [41600/60000 (69%)]	Loss: 0.011853
Train Epoch: 9 [42240/60000 (70%)]	Loss: 0.036105
Train Epoch: 9 [42880/60000 (71%)]	Loss: 0.224044
Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.051845
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.070274
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.035267
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.028523
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.018257
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.071514
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.013804
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.018738
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.009238
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.061489
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.051740
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.018773


In [ ]:
torch.optim.Optimizer

In [ ]:
type(optimizer)

In [ ]:
isinstance(optimizer, torch.optim.SGD)

In [ ]:
issubclass(type(optimizer), torch.optim.Optimizer)